In [4]:
# -*- coding: utf-8 -*-
"""
Created on Tue May 11 2021

@author: Zsuzsanna Koczor-Benda, UCL
"""

from __future__ import print_function
import numpy as np
import os
from urllib.parse import parse_qs,urlparse
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import pandas as pd
from IPython.core.display import HTML
import ipywidgets as widgets
from ipywidgets import HBox, Layout,VBox
from ipywidgets  import interactive_output
from tools.load_database import get_intensities_all,get_intens_range,get_target

In [5]:
# Get url and variables
server = os.environ.get('SERVER_NAME', 'localhost') 
url = "http://" + server
port = os.environ.get('SERVER_PORT', '')
if port:
    url += ":" + port
path = os.environ.get('SCRIPT_NAME', '')
if path:
    url += path
else:
    url="http://localhost:8888/voila/render/Documents/Work/Work_website/molecular-vibration-explorer/database.ipynb"
url="/".join(url.split("/")[:-1])
try:
    query = os.environ.get('QUERY_STRING', '')
    parameters = parse_qs(query)
    SH=int(parameters.get('SH')[0])
except:
    SH=0

In [ ]:
index_url="<a href="+ url+ "/index.ipynb>Go back to Index</a>"
display(HTML(index_url))

In [6]:
# Load all precalculated data for Gold or Thiol database
freqs0,IR,Raa,Rab,Paaa,Pbaa,Paab,Pabc,smiles,fname=get_intensities_all(sclf=0.98,SH=SH)

## Distribution of target property values in the database

In [7]:
# Distribution of P,A, or R for range 30-1000 cm-1 is centered at 0 with std=1 for random set of molecules
# Current dataset contains 1.3k random molecules + 1.5k molecules selected based on ML predictions
    
def calc_target(target_type,pol,tmin,tmax,sclf) :
    if target_type=="P":
        if pol=='aaa':
            intens=Paaa
        elif pol=='abc':
            intens=Pabc
        elif pol=='baa':
            intens=Pbaa
        elif pol=='aab':
            intens=Paab
    elif target_type=="R":
        if pol=='aaa' or pol=='baa':
            intens=Raa
        else:
            intens=Rab
    elif target_type=="A":
        intens=IR
        
    freqs=freqs0*sclf

    intens_range=get_intens_range(freqs,intens,tmin,tmax)      # get intensities in freq range
    target=get_target(intens_range,target_type)                # get chosen target for freq range

    return freqs,intens_range,target

def plot_histogram(target_type,trange,sclf,rb_ir_beam,rb_rin_beam,rb_rout_beam):
    tmin=trange[0]
    tmax=trange[1]

    # Set up field polarization vectors  
    polarizations=[np.array([1.,0.,0.]),np.array([0.,1.,0.]),np.array([0.,0.,1.])]
    e=polarizations[rb_ir_beam]
    e_in=polarizations[rb_rin_beam]
    e_out=polarizations[rb_rout_beam]   
    ir_r_in=int(np.dot(e,e_in))
    ir_r_out=int(np.dot(e,e_out))
    r_in_r_out=int(np.dot(e_in,e_out))
    if ir_r_in and r_in_r_out:
        pol='aaa'
    elif ir_r_in==0 and r_in_r_out==0 and ir_r_out==0:
        pol='abc'
    elif r_in_r_out:
        pol='baa'
    else:
        pol='aab'
    
    freqs,intens_range,target=calc_target(target_type,pol,tmin,tmax,sclf)
    

    if np.isinf(target).any():    # if there are molecules with 0 intensity in range, target will be -infinity
        print("Number of molecules with no modes in range: ",len(target[target < -1E2]))
    ax.hist(target[target > -1E2],20,width=0.25,color='k')  # skip molecules with too small target value
    ax.set_xlabel("{} for range {}-{} ".format(target_type,tmin,tmax)+r'cm$^{-1}$')
    
def on_value_change(change):
    tmin=trange.value[0]
    tmax=trange.value[1]

    # Set up field polarization vectors  
    polarizations=[np.array([1.,0.,0.]),np.array([0.,1.,0.]),np.array([0.,0.,1.])]
    e=polarizations[rb_ir_beam.value]
    e_in=polarizations[rb_rin_beam.value]
    e_out=polarizations[rb_rout_beam.value]   
    ir_r_in=int(np.dot(e,e_in))
    ir_r_out=int(np.dot(e,e_out))
    r_in_r_out=int(np.dot(e_in,e_out))
    if ir_r_in and r_in_r_out:
        pol='aaa'
    elif ir_r_in==0 and r_in_r_out==0 and ir_r_out==0:
        pol='abc'
    elif r_in_r_out:
        pol='baa'
    else:
        pol='aab'
    
    freqs,intens_range,target=calc_target(target_type.value,pol,tmin,tmax,sclf.value)
    
    plt.cla()
    plot_histogram(target_type.value,trange.value,sclf.value,rb_ir_beam.value,rb_rin_beam.value,rb_rout_beam.value)


In [9]:
def path_to_image_html(path):
    return '<img src="'+ path + '" width="200" >'

def show_table(target_type,trange,sclf,page,rb_ir_beam,rb_rin_beam,rb_rout_beam):
    tmin=trange[0]
    tmax=trange[1]
    
    # Set up field polarization vectors  
    polarizations=[np.array([1.,0.,0.]),np.array([0.,1.,0.]),np.array([0.,0.,1.])]
    e=polarizations[rb_ir_beam]
    e_in=polarizations[rb_rin_beam]
    e_out=polarizations[rb_rout_beam]   
    ir_r_in=int(np.dot(e,e_in))
    ir_r_out=int(np.dot(e,e_out))
    r_in_r_out=int(np.dot(e_in,e_out))
    if ir_r_in and r_in_r_out:
        pol='aaa'
    elif ir_r_in==0 and r_in_r_out==0 and ir_r_out==0:
        pol='abc'
    elif r_in_r_out:
        pol='baa'
    else:
        pol='aab'
        
    freqs1,intens_range1,P=calc_target("P",pol,tmin,tmax,sclf)
    freqs2,intens_range2,A=calc_target("A",pol,tmin,tmax,sclf)
    freqs3,intens_range3,R=calc_target("R",pol,tmin,tmax,sclf)

    if target_type=="P":
        target=P
    elif target_type=="R":
        target=R
    elif target_type=="A":
        target=A
        
    nummols=10
    nmols=len(smiles)
    minmol=max(0,nmols-(page)*nummols)
    maxmol=nmols-(page-1)*nummols

    bestmols=list(reversed(np.argsort(target)[minmol:maxmol]))
    molnums=np.arange(nmols-maxmol+1,nmols-minmol+1)
    pd.options.display.float_format = '{:.2f}'.format
    

    images = []
    mollinks=[]
    modelinks=[]
    codes=[]
    for m in pd.Series(bestmols,dtype='int'):
        molcode=(fname[m]).split("-",1)[1]
        codes.append(molcode)
        if SH:
            images.append("data_SH/{}.png".format(molcode))
        else:
            images.append("data_SAu/{}.png".format(molcode))
        molurl="<a href="+ url+ "/molecule.ipynb?Molcode="+molcode+"&SH={}".format(SH)+">Go to molecule page</a>"
        modeurl="<a href="+url+"/modes.ipynb" +"?Molcode="+molcode+"&SH={}".format(SH)+">Check normal modes</a>"
        mollinks.append(molurl)
        modelinks.append(modeurl)

    df = pd.DataFrame({
        'Index' : pd.Series(molnums,dtype='int'),
        'Code': pd.Series(codes),
        'Molecule': pd.Series(images),
        'P' : pd.Series(P[bestmols],dtype='f4'),
        'A' : pd.Series(A[bestmols],dtype='f4'),
        'R' : pd.Series(R[bestmols],dtype='f4'),
        'Link to molecule page': pd.Series(mollinks),
        'Link to normal mode page': pd.Series(modelinks)
        })

    s = df.style.format(formatter=dict(Molecule=path_to_image_html,P="{:.2f}",A="{:.2f}",R="{:.2f}"))
    cell_hover = {  # for row hover use <tr> instead of <td>
    'selector': 'td:hover',
    'props': [('background-color', '#ffffb3')]
    }
    index_names = {
        'selector': '.index_name',
        'props': 'font-style: italic; color: darkgrey; font-weight:normal;'
    }
    headers = {
        'selector': 'th:not(.index_name)',
        'props': 'background-color: grey; color: white;'
    }
    s.set_table_styles([cell_hover, index_names, headers])

    s.set_properties(**{'background-color': 'white',
                           'color': 'black',
                           'border-color': 'black',
                        'border-style':'solid',
                        'border-width':'1.5px',
                       })
    display(HTML(s.hide_index().render()))

In [10]:
%matplotlib widget

# create widgets
layout = widgets.Layout(width='350px', description_width='300px') #set width and height

output_plot = widgets.Output()

target_type=widgets.RadioButtons(options=[['Conversion (P)','P'], ['Absorption (A)','A'],['Raman (R)','R']],
                                          description='Target property:',
                                          style={'description_width': 'initial'},
                                          disabled=False)

trange=widgets.FloatRangeSlider(
                    min=0,
                    max=3900,
                    value=[600,1700],
                    step=1,
                    continuous_update=False,
                    orientation='horizontal',
                    description='Frequency range /cm$^{-1}$:',            
                    layout=layout,
                    style={'description_width': 'initial'})

sclf=widgets.FloatSlider(value=0.98,min=0.9,max=1.1,step=0.01,description='Frequency scaling factor:',
                                  continuous_update=False,
                                  disabled=False,style={'description_width': 'initial'})


axes=[["x",0], ["y",1] ,["z",2]]
layout_rb=widgets.Layout(height='70px',width='160px')
style = {'description_width': '120px'}
rb_ir_beam=widgets.RadioButtons(
                        options=axes,
                        value=2,
                        description='THz/IR beam:',
                        layout=layout_rb,
                        style=style
                    )
rb_rin_beam=widgets.RadioButtons(
                        options=axes,
                        value=2,
                        description=r'Raman $\it{in}$ beam:',
                        layout=layout_rb,
                        style=style
                    )
rb_rout_beam=widgets.RadioButtons(
                        options=axes,
                        value=2,
                        description=r'Raman $\it{out}$ beam:',
                        layout=layout_rb,
                        style=style
                    )


# create plot
tmin=trange.value[0]
tmax=trange.value[1]

# Set up field polarization vectors  
polarizations=[np.array([1.,0.,0.]),np.array([0.,1.,0.]),np.array([0.,0.,1.])]
e=polarizations[rb_ir_beam.value]
e_in=polarizations[rb_rin_beam.value]
e_out=polarizations[rb_rout_beam.value]   
ir_r_in=int(np.dot(e,e_in))
ir_r_out=int(np.dot(e,e_out))
r_in_r_out=int(np.dot(e_in,e_out))
if ir_r_in and r_in_r_out:
    pol='aaa'
elif ir_r_in==0 and r_in_r_out==0 and ir_r_out==0:
    pol='abc'
elif r_in_r_out:
    pol='baa'
else:
    pol='aab'
    
freqs,intens_range,target=calc_target(target_type.value,pol,tmin,tmax,sclf.value)

with output_plot:
    fig, ax = plt.subplots(constrained_layout=True,num='Histogram')
    fig.set_size_inches(4, 3)

if np.isinf(target).any():    # if there are molecules with 0 intensity in range, target will be -infinity
    print("Number of molecules with no modes in range: ",len(target[target < -1E2]))
ax.hist(target[target > -1E2],20,width=0.25,color='k')  # skip molecules with too small target value 
ax.set_xlabel("{} for range {}-{} ".format(target_type.value,tmin,tmax)+r'cm$^{-1}$')
ax.set_ylabel("counts")
plt.rcParams.update({'font.size': 10})


# connect callbacks
target_type.observe(on_value_change, 'value')
trange.observe(on_value_change, 'value')
sclf.observe(on_value_change, 'value')
rb_ir_beam.observe(on_value_change, 'value')
rb_rin_beam.observe(on_value_change, 'value')
rb_rout_beam.observe(on_value_change, 'value')


# create HTML table
nummols=10
pages=np.arange(1,round(len(smiles)/nummols+1))

page=widgets.Dropdown(options=pages,
    value=1,
    description='Page:',layout=widgets.Layout(width='150px',description_width='100px')
)

title=widgets.Label(value="Top molecules")

interactive_table=interactive_output(show_table,
        {'target_type': target_type,
         'trange': trange,
         'sclf' : sclf,
         'page': page,
         'rb_ir_beam':rb_ir_beam,
         'rb_rin_beam':rb_rin_beam,
         'rb_rout_beam':rb_rout_beam
         });


# display widgets
output_plot.layout.height = '400px'
output_plot.layout.width = '460px'
controls=VBox([target_type,trange,sclf,rb_ir_beam,rb_rin_beam,rb_rout_beam])
controls.layout.height = '400px'
display(HBox([output_plot,controls]))
display(VBox([HBox([title,page]),interactive_table]))

ModuleNotFoundError: No module named 'ipympl'